In [1]:
# k crossover d，close > ema 8, ema 8 > ema 14, ema 14 > 50, take profit 2 atr, stop loss 3 atr

In [2]:
import websocket
import requests
import json
import pandas as pd
import datetime as dt
import numpy as np
import time
import ta
from dotenv import load_dotenv
import os
from binance.client import Client
from IPython.display import clear_output
import winsound

In [3]:
frequency = 250  # Set Frequency To 2500 Hertz
duration = 300  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [4]:
load_dotenv()

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

spot = client.get_account()
s_bal = pd.DataFrame(spot['balances'])
print(s_bal)


    asset        free      locked
0     BTC  0.00000000  0.00000000
1     LTC  0.00000000  0.00000000
2     ETH  0.00000000  0.00000000
3     NEO  0.00000000  0.00000000
4     BNB  0.24622450  0.00000000
..    ...         ...         ...
529   GNS  0.00000000  0.00000000
530   SYN  0.00000000  0.00000000
531  LQTY  0.00000000  0.00000000
532    ID  0.00000000  0.00000000
533   ARB  0.00000000  0.00000000

[534 rows x 3 columns]


In [5]:
timezone = 8
endpoint = 'wss://stream.binance.com:9443/ws'
symbol = 'ethusdt'
symbol_C = symbol.upper()
interval = '1h'

# start epoch till now
start_time = 1672586510000
end_time = round(time.time() * 1000)

# step between timestamps in milliseconds, 60000 = 1min 
step = 60000 * 3600

In [6]:
data = json.dumps({'method':'SUBSCRIBE','params':[symbol + '@kline_' + interval],'id':1})

In [7]:
# 先抓歷史資料好讓技術指標能成型
def get_historical(symbol, interval, start_time, end_time, step):
    
    rawdf = pd.DataFrame()
    
    url = "https://api.binance.com/api/v3/klines"
    
    for timestamp in range(start_time, end_time, step):
        params = {"symbol": symbol_C,
                  "interval": interval,
                  "startTime": timestamp,
                  "endTime": timestamp + step}
        response = requests.get(url, params=params).json()
        out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                               "Volume", "Close_Time", "Quote asset volume",
                                               "Number of trades", "Taker buy base asset volume",
                                               "Taker buy quote asset volume", "Ignore"])
        rawdf = pd.concat([rawdf, out], axis = 0)
    
    rawdf = rawdf[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]
    convert_dict = {'Close_Time': float, 'Open': float, 'Close': float, "High": float, "Low": float, 'Volume': float}
    rawdf = rawdf.astype(convert_dict)

    rawdf['Close_Time'] = pd.to_datetime(rawdf['Close_Time'], unit = 'ms')
    rawdf['Close_Time'] = rawdf['Close_Time'] + pd.Timedelta(hours=timezone)
    rawdf['Close_Time'] = rawdf['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    rawdf = rawdf.reset_index(drop=True)
    
    return rawdf

In [8]:
rawdf = get_historical(symbol, interval, start_time, end_time, step)
print(rawdf)

               Close_Time     Open    Close     High      Low      Volume
0     2023-01-02 00:59:59  1196.81  1197.34  1198.35  1195.88   8667.6825
1     2023-01-02 01:59:59  1197.35  1200.68  1203.50  1197.34  10429.0438
2     2023-01-02 02:59:59  1200.67  1200.92  1202.91  1200.67   5269.1583
3     2023-01-02 03:59:59  1200.93  1201.78  1204.51  1200.92   6650.6305
4     2023-01-02 04:59:59  1201.77  1201.40  1204.67  1201.39   4423.7644
...                   ...      ...      ...      ...      ...         ...
1983  2023-03-25 16:59:59  1747.26  1743.38  1750.27  1736.05  18925.7854
1984  2023-03-25 17:59:59  1743.38  1748.17  1750.38  1739.98  12748.7509
1985  2023-03-25 18:59:59  1748.17  1747.45  1750.61  1741.72   9940.0383
1986  2023-03-25 19:59:59  1747.45  1745.25  1750.52  1743.20   7717.0059
1987  2023-03-25 20:59:59  1745.24  1743.58  1746.48  1738.78   9081.2051

[1988 rows x 6 columns]


In [9]:
def indicators(df):

# ema
    for i in (8, 14, 50):
        df['ema_'+str(i)] = ta.trend.ema_indicator(df.Close, window=i)

# atr
    df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)

# kd
    kd_int = 14
    d_int = 3
   
    kddf = pd.DataFrame()
    kddf[str(kd_int) + '-Low'] = df['Low'].rolling(kd_int).min()
    kddf[str(kd_int) + '-High'] = df['High'].rolling(kd_int).max()
    df['slowk'] = (df['Close'] - kddf[str(kd_int) + '-Low'])*100/(kddf[str(kd_int) + '-High'] - kddf[str(kd_int) + '-Low'])
    df['slowd'] = df['slowk'].rolling(d_int).mean()
    
    return df

def checkcross(df):
    series = df['slowk'] > df['slowd']
    return series.diff() & series

In [10]:
def conditions(df):
    # k crossover d，close > ema 8, ema 8 > ema 14, ema 14 > 50, take profit 2 atr, stop loss 3 atr
    df['kd_cross'] = checkcross(df)

    for index, row in df.iterrows():
        # c1
        df['c1'] = df['kd_cross']
        # c2
        df['c2'] = df['Close'] >= df['ema_8']
        # c3
        df['c3'] = df['ema_8'] >= df['ema_14']
        # c4
        df['c4'] = df['ema_14'] >= df['ema_50']

    # 條件達成
    df['signal'] = False
    df.loc[df.c1 & df.c2 & df.c3 & df.c4, 'signal'] = True
            
    # 下一根進場
    df['openbuy'] = False

    return df

In [11]:
def enter_position(df):
    global symbol_C
    
    #-----Calculate entry price-----#
    close_val = df['Close']
    atr_val = df['atr']
    df.loc[df.index[-1], 'entry_p'] = close_val.loc[close_val.index[-2]]
    df.loc[df.index[-1], 'stop_loss'] = close_val.loc[close_val.index[-2]] - 3 * atr_val.loc[atr_val.index[-2]]
    df.loc[df.index[-1], 'take_profit'] = close_val.loc[close_val.index[-2]] + 3.5 * atr_val.loc[atr_val.index[-2]]

    #-----position attributes-----#
    quantity = 0.1
    entry_p = df.loc[df.index[-2], 'Close']
    stop_loss_p = df.loc[df.index[-1], 'stop_loss']
    take_profit_p = df.loc[df.index[-1], 'take_profit']


    print(df.loc[df.index[-1], 'entry_p'] == entry_p)  
    
    try:
        order = client.futures_create_order(
            symbol=symbol_C,
            side='BUY',
            type='LIMIT',
            timeInForce='GTC',
            quantity=quantity,
            price=entry_p,
            positionSide='LONG',
        )
        print('Order created successfully.')       
        
        stop_loss_order = client.futures_create_order(
            symbol=symbol_C,
            side='SELL',
            type='STOP_MARKET',
            stopPrice=stop_loss_p,
            stopLimitTimeInForce='GTC',
            closePosition = 'true',
            positionSide = 'LONG'

        )
        print('Stop loss created successfully')
        
        take_profit_order = client.futures_create_order(
            symbol=symbol_C,
            side='SELL',
            type='TAKE_PROFIT_MARKET',
            stopPrice=take_profit_p,
            stopLimitTimeInForce='GTC',
            closePosition = 'true',
            positionSide = 'LONG'
        )
        winsound.Beep(frequency, duration)
        print('Take profit created successfully')
            
    except Exception as e:
        print(f'Error creating order: {e}')

    print(str(symbol_C) + ' entered at ' + str(entry_p) + ', stop loss at ' + str(stop_loss_p))


In [12]:
line_count = 0
max_lines = 5

current_k = 0

def on_open(ws):
    ws.send(data)
    
def on_message(ws, message):
    global current_k, line_count, max_lines
    df = get_historical(symbol, interval, start_time, end_time, step)

    indicators(df)
    conditions(df)
    
    try:
        # 如果時間標籤改變，代表上個 Close 發生，檢查是否進場
        if df['Close_Time'][len(df) - 1] != current_k:
            print('time changed')
            current_k = df['Close_Time'][len(df) - 1]
            if df.iloc[df.index[-2]]['signal'] == True:
                winsound.Beep(frequency, duration)
                df.loc[df.index[-1], 'openbuy'] = True
                enter_position(df)
                print(df.tail(2))
        
        # 隨時偵測出場條件是否成立，不必等 Close 發生
#         if ((df.loc[df.index[-1], 'rsi'] >= tp_rsi) & (df.loc[df.index[-1], 'slowk'] >= tp_slowk) & (df.loc[df.index[-1], 'slowd'] >= tp_slowd)):
#             take_profit_order = client.futures_create_order(
#                 symbol=symbol_C,
#                 side='SELL',
#                 type='TAKE_PROFIT_MARKET',
#                 stopPrice=df.loc[df.index[-1], 'Close'],
#                 stopLimitTimeInForce='GTC',
#                 closePosition = 'true',
#                 positionSide = 'LONG'
#             )
#             winsound.Beep(frequency, duration)
#             print('Take profit created successfully')
        
    except Exception as e:
        print(f'Error creating order: {e}')


    try:
        df = df.reset_index(drop=True)
        print('nothing happened...' + str(df.loc[df.index[-1], 'Close_Time']) + ' ' + str(df.loc[df.index[-1], 'Close']))
        print()
        print(df.tail(2))
        print('----------------------------------------------------------------------------')
        
        positions = client.futures_account()['positions']
        for position in positions:
            if float(position['positionAmt']) != 0:
                position_df = pd.DataFrame({'Symbol':position['symbol'],
                                            'Entry_P':round(float(position['entryPrice']),2),
                                            'Amt':round(float(position['positionAmt']) * df.loc[df.index[-1], 'Close'],2),
                                            'PL':round(float(position['unrealizedProfit']),2),
                                            'X':round(float(position['leverage']),1),
                                           }, index=[0])      
                print(position_df)
    
    except Exception as e:
        print(f'Error: {e}')    
        
        
    line_count += 1
    if line_count >= max_lines:
        clear_output(wait=True)
        line_count = 0

In [ ]:
ws = websocket.WebSocketApp(endpoint, on_message = on_message, on_open = on_open)
ws.run_forever()

nothing happened...2023-03-25 20:59:59 1743.85

               Close_Time     Open    Close     High      Low      Volume  \
1986  2023-03-25 19:59:59  1747.45  1745.25  1750.52  1743.20   7717.0059   
1987  2023-03-25 20:59:59  1745.24  1743.85  1746.48  1738.78  10050.9823   

            ema_8       ema_14       ema_50        atr      slowk      slowd  \
1986  1747.824480  1751.010656  1766.672625  12.601337  34.874905  48.409738   
1987  1746.941262  1750.055902  1765.777620  12.251242  29.567854  40.110680   

      kd_cross     c1     c2     c3     c4  signal  openbuy  
1986     False  False  False  False  False   False    False  
1987     False  False  False  False  False   False    False  
----------------------------------------------------------------------------
    Symbol  Entry_P    Amt    PL     X
0  ETHUSDT  1743.27 -226.7  0.04  10.0
nothing happened...2023-03-25 20:59:59 1743.84

               Close_Time     Open    Close     High      Low      Volume  \
1986  2023-03